## Imports

In [ ]:
import imageio
import cv2
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

## Detect and convert image colors

In [ ]:
def threshold_rel(img, lo, hi):
    vmin = np.min(img)
    vmax = np.max(img)
    
    vlo = vmin + (vmax - vmin) * lo
    vhi = vmin + (vmax - vmin) * hi
    return np.uint8((img >= vlo) & (img <= vhi)) * 255

def threshold_abs(img, lo, hi):
    return np.uint8((img >= lo) & (img <= hi)) * 255


def getHLSImage(img): 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    v_channel = hsv[:,:,2]

    right_lane = threshold_rel(l_channel, 0.8, 1.0)
    right_lane[:,:750] = 0

    left_lane = threshold_abs(h_channel, 20, 30)
    left_lane &= threshold_rel(v_channel, 0.7, 1.0)
    left_lane[:,550:] = 0

    finalImg = left_lane | right_lane
    return finalImg


def plotImage(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


## Canny Edge Detection

In [ ]:
def cannyEdgeDetection(img):
    blurredImg = cv2.GaussianBlur(img, (5,5), 0)
    cannyImg = cv2.Canny(blurredImg, 160 ,170)
    return cannyImg

## Image Masking

In [ ]:
def maskImage(img):
    height = img.shape[0]
    width = img.shape[1]
    regionOfInterest = [
        (0, height),
        (width/2, height/2),
        (width, height)
    ]
    rectangle = np.array([regionOfInterest], np.int32)
    mask = np.zeros_like(img)
    # all pixels will be black
    cv2.fillPoly(mask, rectangle, 255)

    # plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
    maskedImage = cv2.bitwise_and(img, mask)
    return maskedImage

## Pipeline

In [ ]:
vid = imageio.get_reader('/content/drive/My Drive/challenge_video.mp4', 'ffmpeg')
finalVid = imageio.get_writer('/content/drive/My Drive/output_challenge_video.mp4', fps = 30)

for i, img in enumerate(vid):
    HLSImg = getHLSImage(img)
    cannyImg = cannyEdgeDetection(HLSImg)
    maskedImage = maskImage(cannyImg)
    finalVid.append_data(maskedImage)


finalVid.close()